# Credit Risk Resampling Techniques

## Deliverable 1

In [97]:
import warnings
warnings.filterwarnings('ignore')

In [98]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [99]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [100]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 86 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   68817 non-null  float64
 1   int_rate                    68817 non-null  float64
 2   installment                 68817 non-null  float64
 3   home_ownership              68817 non-null  object 
 4   annual_inc                  68817 non-null  float64
 5   verification_status         68817 non-null  object 
 6   issue_d                     68817 non-null  object 
 7   loan_status                 68817 non-null  object 
 8   pymnt_plan                  68817 non-null  object 
 9   dti                         68817 non-null  float64
 10  delinq_2yrs                 68817 non-null  float64
 11  inq_last_6mths              68817 non-null  float64
 12  open_acc                    68817 non-null  float64
 13  pub_rec                     688

Create the training variables by converting the string values into numerical ones using the get_dummies() method.
  
Check Dtypes to catch all object/string variables: 'home_ownership', 'verification_status', 'issue_d', 'paymnt_plan', 
    'initial_list_status', 'next_pmnt_d', 'application_type', 'hardship_flag', 'debt_settlement_flag'

In [102]:
df_binary_encoded = pd.get_dummies(df, columns=[
                        'initial_list_status',
                        'home_ownership',
                        'verification_status',
                        'issue_d',
                        'pymnt_plan',
                        'next_pymnt_d',
                        'application_type',
                        'hardship_flag', 
                        'debt_settlement_flag'])
df_binary_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,pymnt_plan_n,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0,0,1,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,0,0,1,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,0,0,1,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,0,0,1,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,0,0,1,1,0,1,1,0,1,1


# Split the Data into Training and Testing

In [103]:
# Create our features
X = df_binary_encoded.drop(columns="loan_status", axis=1)

# Create our target
y = df["loan_status"]
y

0        low_risk
1        low_risk
2        low_risk
3        low_risk
4        low_risk
           ...   
68812    low_risk
68813    low_risk
68814    low_risk
68815    low_risk
68816    low_risk
Name: loan_status, Length: 68817, dtype: object

In [104]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [105]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(51612, 95)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

What is happening below:

1. RandomOversampler is being imported
2. An instance of RandomOverSampler is instantiated as ros.
3. The training data (X_train and y_train) is resampled using the fit_resample() method.
4. The results are called X_resampled and y_resampled.
5. Using counter to show the train data has been resampled


In [197]:
X_train.shape

(51612, 95)

In [199]:
## concern on resampling!
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [200]:
X_resampled.shape

(102704, 95)

In [201]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=1)
#model = LogisticRegression(solver='lbfgs',max_iter=200, random_state=1)

#Fit the model to resampled data
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [202]:
# Make Predictions
y_pred = model.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,low_risk,low_risk
1,high_risk,low_risk
2,high_risk,low_risk
3,high_risk,low_risk
4,high_risk,low_risk
5,high_risk,low_risk
6,high_risk,low_risk
7,low_risk,low_risk
8,low_risk,low_risk
9,low_risk,low_risk


In [203]:
# Calculated the balanced accuracy score
from sklearn.metrics import accuracy_score
acc_score = (accuracy_score(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

0.6559721011333914


In [204]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report

matrix = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    matrix, index=["Actual High-Risk", "Actual Low-Risk"], columns=["Predicted High_Risk", "Predicted Low_Risk"])
cm_df


,Predicted High_Risk,Predicted Low_Risk
Actual High-Risk,55,32
Actual Low-Risk,5887,11231


In [205]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.63      0.66      0.02      0.64      0.41        87
   low_risk       1.00      0.66      0.63      0.79      0.64      0.42     17118

avg / total       0.99      0.66      0.63      0.79      0.64      0.42     17205



In [206]:
# Displaying results; RandomOverSampler
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))

Confusion Matrix


,Predicted High_Risk,Predicted Low_Risk
Actual High-Risk,55,32
Actual Low-Risk,5887,11231


Accuracy Score : 0.6559721011333914
Classification Report
              precision    recall  f1-score   support

   high_risk       0.01      0.63      0.02        87
    low_risk       1.00      0.66      0.79     17118

    accuracy                           0.66     17205
   macro avg       0.50      0.64      0.40     17205
weighted avg       0.99      0.66      0.79     17205



### SMOTE Oversampling

In [198]:
X_train.shape

(51612, 95)

In [212]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resample2, y_resample2 = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)
#X_resample2, y_resample2 = SMOTE(random_state=1).fit_resample(X_train, y_train)

In [213]:
Counter(y_resample2)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [214]:
X_resample2.shape

(102704, 95)

In [215]:
# Train the Logistic Regression model using the resampled data
#model = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)
model = LogisticRegression(random_state=1)

model.fit(X_resample2, y_resample2)

y_pred_sm = model.predict(X_test)

In [216]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
acc_score2 = balanced_accuracy_score(y_test, y_pred_sm)
acc_score2

0.6346384729121595

In [217]:
# Display the confusion matrix
matrix_sm = confusion_matrix(y_test, y_pred_sm)

cm2_df = pd.DataFrame(
    matrix_sm, index=["Actual High-Risk", "Actual Low-Risk"], columns=["Predicted High_Risk", "Predicted Low_Risk"])
cm2_df

,Predicted High_Risk,Predicted Low_Risk
Actual High-Risk,56,31
Actual Low-Risk,6409,10709


In [166]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_sm))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.64      0.63      0.02      0.63      0.40        87
   low_risk       1.00      0.63      0.64      0.77      0.63      0.40     17118

avg / total       0.99      0.63      0.64      0.77      0.63      0.40     17205



In [181]:
# Displaying results; SMOTE
print("Confusion Matrix")
display(cm2_df)
print(f"Accuracy Score : {acc_score2}")
print("Classification Report")
print(classification_report(y_test, y_pred_sm))

Confusion Matrix


,Predicted High_Risk,Predicted Low_Risk
Actual High-Risk,56,31
Actual Low-Risk,6409,10709


Accuracy Score : 0.6346384729121595
Classification Report
              precision    recall  f1-score   support

   high_risk       0.01      0.64      0.02        87
    low_risk       1.00      0.63      0.77     17118

    accuracy                           0.63     17205
   macro avg       0.50      0.63      0.39     17205
weighted avg       0.99      0.63      0.77     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [169]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resample3, y_resample3 = cc.fit_resample(X_train, y_train)
Counter(y_resample3)

Counter({'high_risk': 260, 'low_risk': 260})

In [176]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
#model = LogisticRegression(solver='lbfgs', random_state=1)

model = LogisticRegression(random_state=1)

model.fit(X_resample3, y_resample3)

y_pred_cc = model.predict(X_test)

In [184]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

acc_score3 = balanced_accuracy_score(y_test, y_pred_cc)
acc_score3

0.5148442252760757

In [185]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
matrix_cc = confusion_matrix(y_test, y_pred_cc)

cm3_df = pd.DataFrame(
    matrix_cc, index=["Actual High-Risk", "Actual Low-Risk"], columns=["Predicted High_Risk", "Predicted Low_Risk"])
cm3_df

,Predicted High_Risk,Predicted Low_Risk
Actual High-Risk,53,34
Actual Low-Risk,9920,7198


In [186]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_cc))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.61      0.42      0.01      0.51      0.26        87
   low_risk       1.00      0.42      0.61      0.59      0.51      0.25     17118

avg / total       0.99      0.42      0.61      0.59      0.51      0.25     17205



In [187]:
# Displaying results; ClusterCentroids
print("Confusion Matrix")
display(cm3_df)
print(f"Accuracy Score : {acc_score3}")
print("Classification Report")
print(classification_report(y_test, y_pred_cc))

Confusion Matrix


,Predicted High_Risk,Predicted Low_Risk
Actual High-Risk,53,34
Actual Low-Risk,9920,7198


Accuracy Score : 0.5148442252760757
Classification Report
              precision    recall  f1-score   support

   high_risk       0.01      0.61      0.01        87
    low_risk       1.00      0.42      0.59     17118

    accuracy                           0.42     17205
   macro avg       0.50      0.51      0.30     17205
weighted avg       0.99      0.42      0.59     17205



# Deliverable 2
# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [188]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resample4, y_resample4 = smote_enn.fit_resample(X, y)


In [189]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
#model = LogisticRegression(solver='lbfgs', random_state=1)
model = LogisticRegression(random_state=1)

model.fit(X_resample4, y_resample4)

from sklearn.metrics import confusion_matrix
y_pred_st = model.predict(X_test)


In [190]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

acc_score4 = balanced_accuracy_score(y_test, y_pred_st)
acc_score4

0.6464691331165823

In [191]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
matrix_st = confusion_matrix(y_test, y_pred_st)

cm4_df = pd.DataFrame(
    matrix_st, index=["Actual High-Risk", "Actual Low-Risk"], columns=["Predicted High_Risk", "Predicted Low_Risk"])
cm4_df

,Predicted High_Risk,Predicted Low_Risk
Actual High-Risk,60,27
Actual Low-Risk,6791,10327


In [192]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_st))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.69      0.60      0.02      0.65      0.42        87
   low_risk       1.00      0.60      0.69      0.75      0.65      0.41     17118

avg / total       0.99      0.60      0.69      0.75      0.65      0.41     17205



In [193]:
# Displaying results; SMOTEENN
print("Confusion Matrix")
display(cm4_df)
print(f"Accuracy Score : {acc_score4}")
print("Classification Report")
print(classification_report(y_test, y_pred_st))

Confusion Matrix


,Predicted High_Risk,Predicted Low_Risk
Actual High-Risk,60,27
Actual Low-Risk,6791,10327


Accuracy Score : 0.6464691331165823
Classification Report
              precision    recall  f1-score   support

   high_risk       0.01      0.69      0.02        87
    low_risk       1.00      0.60      0.75     17118

    accuracy                           0.60     17205
   macro avg       0.50      0.65      0.38     17205
weighted avg       0.99      0.60      0.75     17205

